In [1]:
from rtf_utils.moat_utils import MoatTile
import json
import logging
import sys
import os
from rtf_utils.gcp_utils import CloudStorage,BigQuery
from essence.analytics.platform import securedcredentials as secure_creds

In [2]:
service_account_email='131786951123-compute@developer.gserviceaccount.com' ## gcp analytics service account
PROJECT_ID='essence-analytics-dwh'
credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)


moat_token = "IhSsMXBfH1eQBYaeYEYDJxot3Gs0uq9iaIUm3Qzf"
start_date = '20190801'
end_date = '20190831'

C:\Users\kyle.randolph\AppData\Local\Continuum\anaconda3\lib\site-packages\google\auth\_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Moat Stuff

In [3]:
ig_vid = MoatTile(tile_id=13120,
         tile_type="vid",name = "ig_vid",
         level_filters={'level1':10154328017481183},
         dimensions=['date','level4'])

yt_trv = MoatTile(tile_id=13386,
         tile_type="vid",name = "YT_TRV",
         level_filters={'level1':7020493427},
         dimensions=['date','level4'])
"""

cm_disp = MoatTile(tile_id=2506,
         tile_type="disp",name = "cm_disp",
         level_filters={'level1':22830945},
         dimensions=['date','level3'])
                   
cm_vid = MoatTile(tile_id=2698,
         tile_type="vid",name = "cm_vid",
         level_filters={'level1':22830945},
         dimensions=['date','level3'])
"""

gcs = CloudStorage(credentialsFromVault)
bq = BigQuery(credentialsFromVault)

def request_and_upload(moat_tile,dest_bucket,folder=None):
    filename = moat_tile.get_data(start_date, end_date,moat_token)
    if folder:
        blob_name = folder + "/" + filename
    
    gcs.upload_blob(dest_bucket,blob_name,filename,mode='filename')
    
    gcs_uri = "gs://" + dest_bucket + "/" + blob_name
    
    print("File Upload to {}".format(gcs_uri))
    
    os.remove(filename)
    print("{} Removed Locally".format(filename))
    
    return gcs_uri

In [4]:
queue = [(ig_vid,"rtf_staging","bonitoQ3"),(yt_trv,"rtf_staging","bonitoQ3")]
uris = [request_and_upload(*x) for x in queue] ##unpack tuple of arg

Data Available, Storing
File Upload to gs://rtf_staging/bonitoQ3/13120_ig_vid.json
13120_ig_vid.json Removed Locally
Data Available, Storing
File Upload to gs://rtf_staging/bonitoQ3/13386_YT_TRV.json
13386_YT_TRV.json Removed Locally


In [5]:
bq.load_from_gcs('rtf_brand_reporting',
                 'gs://rtf_staging/bonitoQ3/13386_YT_TRV.json',
                 '01_MOAT_TRV',mode='Truncate',ext='json')

In [6]:
bq.load_from_gcs('rtf_brand_reporting',
                 'gs://rtf_staging/bonitoQ3/13120_ig_vid.json',
                 '01_MOAT_IG_VID',mode='Truncate',ext='json')

## ------ DCM Section ------

In [5]:
ig_vid.__dict__

{'brandid': 13120,
 'tile_type': 'vid',
 'name': 'ig_vid',
 'filters': {'level1': 10154328017481183},
 'last_request_time': None,
 'time_since_last_request': None,
 'dimensions': ['date', 'level4'],
 'metrics': ['impressions_analyzed',
  'susp_human',
  'human_and_viewable',
  'susp_bot_geo_perc',
  'player_vis_and_aud_on_complete_sum',
  'player_audible_on_complete_sum',
  'player_visible_on_complete_sum',
  'reached_first_quart_sum',
  'reached_second_quart_sum',
  'reached_third_quart_sum',
  'reached_complete_sum',
  'avg_real_estate_perc',
  'player_audible_full_vis_half_time_sum',
  '5_sec_in_view_impressions',
  'susp_human_and_inview_gm_meas_sum']}

#### DCM ToDo:
- refactor report class take same data range format. Declare one format globally and format on class instantiation
- Put dcm report cleaner in class
- put report wait-until-available in class
- figure out a better way to materalize views -- storing sql etc (use parameters for tables names

In [14]:
from rtf_utils.dfa_utils import CampaignManagerReport
import time

In [18]:
def report_workflow(dcm_report):
    print(dcm_report.report['name'])
    dcm_report.set_date_range("2019-07-01","2019-08-25")
    dcm_report.run_report()
    
    dcm_report.get_report_status()
    
    while dcm_report.status != "REPORT_AVAILABLE":
        time.sleep(10)
        dcm_report.get_report_status()
        print(dcm_report.status)
        
    file_name = report.download_file()
    
    report.clean_dcm_file()

In [19]:
gcs = CloudStorage(credentialsFromVault)
verification_report = CampaignManagerReport(credentialsFromVault,5096586,631165400)
video_player_size = CampaignManagerReport(credentialsFromVault,5096586,632709822)

In [20]:
result = map(report_workflow,[verification_report,video_player_size])

In [24]:
verification_report.__dict__


{'profile_id': 5096586,
 'report_id': 631165400,
 'service': <googleapiclient.discovery.Resource at 0x1d389c1dcf8>,
 'report': {'kind': 'dfareporting#report',
  'id': '631165400',
  'etag': '"mD-6LEJOy1ZTRjP0qj4BvgI6FwY/2AdNSY0xLOj384n4Ov8RCfxpBYI"',
  'lastModifiedTime': '1567107974000',
  'ownerProfileId': '5197281',
  'accountId': '5295',
  'name': 'Bonito Classification',
  'fileName': 'Bonito_Classification',
  'format': 'CSV',
  'type': 'STANDARD',
  'criteria': {'dateRange': {'kind': 'dfareporting#dateRange',
    'startDate': '2019-07-01',
    'endDate': '2019-08-25'},
   'dimensions': [{'kind': 'dfareporting#sortedDimension',
     'name': 'dfa:campaign'},
    {'kind': 'dfareporting#sortedDimension', 'name': 'dfa:campaignId'},
    {'kind': 'dfareporting#sortedDimension',
     'name': 'dfa:contentClassification'},
    {'kind': 'dfareporting#sortedDimension', 'name': 'dfa:placement'},
    {'kind': 'dfareporting#sortedDimension', 'name': 'dfa:date'}],
   'metricNames': ['dfa:verifi

In [9]:
materialize_verification = """
                            SELECT
                              date,
                              CAST(REGEXP_EXTRACT(placement,r"OPID-(\d+)") AS int64) AS opid,
                              SUM(verifiable_impressions) verifiable_impressions_no_issues
                            FROM
                              `essence-analytics-dwh.rtf_brand_reporting.DCM_contentVerification`
                            WHERE
                              classifier = "None Detected"
                            GROUP BY
                              1,
                              2

                            """

materialize_player_report = """
                            WITH
                              player_pivot AS (
                              SELECT
                                date,
                                CAST(REGEXP_EXTRACT(placement,r"OPID-(\d+)") AS int64) AS opid,
                                CASE
                                  WHEN video_player_size = "HD" THEN verifiable_impressions
                              END
                                AS hd_player_impressions,
                                CASE
                                  WHEN video_player_size = "LARGE" THEN verifiable_impressions
                              END
                                AS large_player_impressions,
                                CASE
                                  WHEN video_player_size = "SMALL" THEN verifiable_impressions
                              END
                                AS small_player_impressions,
                                CASE
                                  WHEN video_player_size = "(not set)" THEN verifiable_impressions
                              END
                                AS player_not_set_impressions
                              FROM
                                `essence-analytics-dwh.rtf_brand_reporting.DCM_playerSize`)
                            SELECT
                              date,
                              opid,
                              SUM(hd_player_impressions) hd_player_impressions,
                              SUM(large_player_impressions) large_player_impressions,
                              SUM(small_player_impressions) small_player_impressions,
                              SUM(player_not_set_impressions) player_not_set_impressions
                            FROM
                              player_pivot
                            GROUP BY
                              1,
                              2
                            """


moat_vid = """
            SELECT
              date,
              CAST(REGEXP_EXTRACT(level4_label,r"OPID-(\d+)") AS int64) AS opid,
              SUM(impressions_analyzed) AS moat_vid_impressions_analyzed,
              SUM(susp_valid) AS moat_vid_susp_valid,
              SUM(valid_and_viewable) AS valid_and_viewable,
              SUM(reached_first_quart_sum) reached_first_quart_sum,
              SUM(reached_second_quart_sum) reached_second_quart_sum,
              SUM(reached_third_quart_sum) reached_third_quart_sum,
              SUM(reached_complete_sum) reached_complete_sum,
              SUM(player_visible_on_complete_sum) player_visible_on_complete_sum,
              SUM(player_audible_on_complete_sum) player_audible_on_complete_sum,
              SUM(player_vis_and_aud_on_complete_sum) player_vis_and_aud_on_complete_sum,
              SUM(susp_valid_and_inview_gm_meas_sum) susp_valid_and_inview_gm_meas_sum,
              sum(_5_sec_in_view_impressions) _5_sec_in_view_impressions,
              sum(susp_bot_geo_perc) susp_bot_geo_perc
            FROM
              `essence-analytics-dwh.rtf_brand_reporting.MOAT_videoTiles_RAW`

            Group by 1,2
            """

moat_disp = """
            SELECT
              date,
              CAST(REGEXP_EXTRACT(level3_label,r"OPID-(\d+)") AS int64) AS opid,
              SUM(impressions_analyzed) AS impressions_analyzed,
              SUM(valid_and_viewable) AS valid_and_viewable,
              SUM(iva) AS iva,
              SUM(susp_bot_geo_perc) AS susp_bot_geo_perc
            FROM
              `essence-analytics-dwh.rtf_brand_reporting.MOAT_displayTiles_RAW`
            GROUP BY
              1,
              2
            """




final_table = """
SELECT
Time_1__Date_of_Activity as date,Olive_Plan_Line_ID as OLIVE_plan_id,
Olive_Plan_Line_Name as OLIVE_plan_line_name,
Olive__Plan_Line_Property_ as OLIVE_plan_line_property,
Olive_Media_Type as OLIVE_media_type,
Performance_And_Delivery_1__Impressions as OLIVE_impressions,
Performance_And_Delivery_2__Clicks as OLIVE_clicks,
Performance_And_Delivery_6__Spend_in_USD as OLIVE_spend_usd,
null as MOAT_disp_impressions_analyzed,
null as MOAT_dis_susp_valid,
null as MOAT_dis_valid_and_viewable,
vid_impressions_analyzed as MOAT_vid_impressions_analyzed,
vid_susp_valid as MOAT_vid_valid_impressions,
vid_valid_and_viewable as MOAT_vid_valid_and_viewable, 
reached_first_quart_sum as MOAT_reached_first_quart_sum,
reached_second_quart_sum as MOAT_reached_second_quart_sum,
reached_third_quart_sum as MOAT_reached_third_quart_sum,
reached_complete_sum as MOAT_reached_complete_sum,
player_visible_on_complete_sum as MOAT_player_visible_on_complete_sum,
player_audible_on_complete_sum as MOAT_player_audible_on_complete_sum ,
player_vis_and_aud_on_complete_sum as MOAT_player_vis_and_aud_on_complete_sum,
susp_valid_and_inview_gm_meas_sum as MOAT_susp_valid_and_inview_gm_meas_sum,
0 + vid_valid_and_viewable as MOAT_valid_and_viewable,
0 + vid_impressions_analyzed as MOAT_total_impressions_analyzed,
0 + vid_susp_valid as MOAT_total_moat_valid_impressions,
null as iva,
Non_US_Impressions as DCM_non_us_impressions,
verifiable_impressions_no_issues as DCM_verification_no_issues,
small_player_impressions as DCM_small_player_impressions, 
large_player_impressions as DCM_large_player_impressions, 
hd_player_impressions as DCM_hd_player_impressions,
Olive_Plan_Name as OLIVE_plan_name
FROM `essence-analytics-dwh.rtf_brand_reporting.V_brandReport_FINAL`

"""

